**Blackjack!**


Blackjack, also known as twenty-one, is a popular card game played in casinos worldwide. The objective of the game is to beat the dealer by having a hand value closer to 21 without going over. Each player is dealt two cards, and then has the option to "hit" (receive another card) or "stand" (keep their current hand). Players can also double their bet or split their hand if they receive two cards of the same value. The game requires a combination of skill and luck, making it an exciting and challenging game for players of all levels.


<div class="markdown-google-sans">

## **Import Packages!**
</div>


In [2]:
from collections import namedtuple
from itertools import product
from random import shuffle
from typing import List

<div class="markdown-google-sans">

## **Creating Classes!**
</div>

First we define our classes which we need for the game.

The Card named tuple is created using the namedtuple function from the collections module. The Card named tuple has two fields, rank and suit, which represent the rank and suit of a playing card, respectively.

The Deck class represents a deck of cards for the blackjack game. It has a class variable card_ranks, which is a list of strings representing the ranks of the playing cards, and a class variable card_suits, which is a list of strings representing the suits of the playing cards.

In the __init__ method, the Deck class initializes an empty list of cards and calls the refresh_deck method to fill the cards list with all the possible combinations of ranks and suits using the product function from the itertools module.

The refresh_deck method creates a list of Card named tuples by iterating over the **Cartesian** product of card_ranks and card_suits using list comprehension.

The shuffle method shuffles the list of cards in place using the shuffle function from the random module.

The draw_card method removes and returns the last card from the cards list using the pop method. It also returns a Card named tuple representing the rank and suit of the drawn card.

In [3]:
Card = namedtuple('Card', ('rank', 'suit'))

class Deck:
    card_ranks = ['Ace', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King']
    card_suits = ['Hearts', 'Spades', 'Clubs', 'Diamonds']

    def __init__(self) -> None:
        self.cards = []
        self.refresh_deck()

    def refresh_deck(self) -> int:
        self.cards = [Card(*card) for card in list(product(self.card_ranks, self.card_suits))]

    def shuffle(self) -> None:
        shuffle(self.cards)

    def draw_card(self) -> Card:
        return self.cards.pop()

The Hand class represents a hand of playing cards for a player or dealer in the blackjack game. It has three instance variables: cards, value, and aces.

In the __init__ method, the Hand class initializes an empty list of cards, sets the initial value of the hand to 0, and sets the initial number of aces to 0.

The card_value method takes a Card named tuple as an argument and returns the value of the card according to the rules of blackjack. The method defines a dictionary values that maps each card rank to its value. For non-Ace cards, the value is simply the rank value. For Ace cards, the value is initially set to 11, but it can be reduced to 1 later if needed.

The add_card method takes a Card named tuple as an argument and adds it to the cards list. It also updates the value of the hand by adding the value of the new card using the card_value method. If the new card is an Ace, the aces counter is incremented.

The adjust_for_ace method adjusts the value of the hand if it contains one or more Aces and the total value of the hand exceeds 21. In this case, each Ace is treated as having a value of 1 instead of 11 by subtracting 10 from the total value of the hand and decrementing the aces counter. This process is repeated until the total value of the hand is less than or equal to 21 or there are no more Aces to adjust.

In [4]:
class Hand:
    def __init__(self) -> None:
        self.cards = []
        self.value = 0
        self.aces = 0
    
    def card_value(self, card: Card) -> int:
        values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'Jack':10,
        'Queen':10, 'King':10, 'Ace':11}
        return values[card]
        
    def add_card(self, card: Card) -> None:
        self.cards.append(card)
        self.value += self.card_value(card[0])
        if card[0] == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self) -> None:
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

The Chips class has two instance variables: total and bet. total represents the total amount of chips that the player has, and bet represents the amount of chips that the player has bet in the current hand.

The __init__ method initializes the total instance variable to 100 and the bet instance variable to 0.

The win_bet method is called when the player wins a hand. It adds the bet amount to the total amount.

The lose_bet method is called when the player loses a hand. It subtracts the bet amount from the total amount.

In [5]:
class Chips:
    def __init__(self) -> None:
        self.total = 100
        self.bet = 0

    def win_bet(self) -> None:
        self.total += self.bet

    def lose_bet(self) -> None:
        self.total -= self.bet

The Game class takes a Deck object as input and uses it to initialize the game. The take_bet method takes a Chips object as input and prompts the player to enter their bet. The hit method takes a Hand object as input and deals a card from the deck to the hand, adjusting the hand's value if it contains an ace. The hit_or_stand method takes a Hand object as input and prompts the player to choose whether to hit or stand. The show_some method takes two Hand objects as input and shows the player one of the dealer's cards and all of the player's cards. The show_all method takes two Hand objects as input and shows all of the dealer's and player's cards and their respective values. The player_busts, player_wins, dealer_busts, dealer_wins, and push methods take two Hand objects and a Chips object as input and print the appropriate message and adjust the player's chips depending on the outcome of the game. Overall, this code provides the main gameplay logic for a blackjack game.

In [6]:
class Game:
    def __init__(self, deck: Deck) -> None:
        self.deck = deck
        self.deck.shuffle()
        self.visibile_cards = []

    def take_bet(self, chips: Chips):
        while True:
            try:
                bet = int(input('How many chips would you like to bet? '))
            except ValueError:
                print('Sorry, a bet must be an integer!')
            else:
                if bet > chips.total:
                    print("Sorry, your bet can't exceed",chips.total)
                else:
                    break
        return bet

    def hit(self, hand):
        hand.add_card(self.deck.draw_card())
        hand.adjust_for_ace()
    
    def hit_or_stand(self, hand):
        global playing
        while True:
            x = input("Hit or Stand? Enter 'h' or 's' ")
            if x[0].lower() == 'h':
                self.hit(hand)
            elif x[0].lower() == 's':
                print("Player stands. Dealer is playing.")
                playing = False
            else:
                print("Sorry, please try again.")
                continue
            break
    
    def show_some(self, player, dealer):
        print("\nDealer's Hand:")
        print(" <card hidden>")
        print(f'{dealer.cards[0][1]} {dealer.cards[0][0]}')
        print("\nPlayer's Hand:", ", ".join([f'{suit} {rank}' for rank, suit in player.cards]), sep='\n ')
    
    def show_all(self, player, dealer):
        print("\nDealer's Hand:", ", ".join([f'{suit} {rank}' for rank, suit in dealer.cards]), sep='\n ')
        print("Dealer's Hand =",dealer.value)
        print("\nPlayer's Hand:", ", ".join([f'{suit} {rank}' for rank, suit in player.cards]), sep='\n ')
        print("Player's Hand =",player.value)
    
    def player_busts(self, player, dealer, chips):
        print("Player busts!")
        chips.lose_bet()
    
    def player_wins(self, player, dealer, chips):
        print("Player wins!")
        chips.win_bet()
    
    def dealer_busts(self, player, dealer, chips):
        print("Dealer busts!")
        chips.win_bet()

    def dealer_wins(self, player, dealer, chips):
        print("Dealer wins!")
        chips.lose_bet()
    
    def push(self, player, dealer):
        print("Dealer and Player tie! It's a push.")

<div class="markdown-google-sans">

## **Gameplay!**
</div>

First, the code defines a Hand class that represents a player's hand in the game. It has attributes for the cards in the hand, the value of the hand, and the number of aces in the hand. It also has methods for calculating the value of a card, adding a card to the hand, and adjusting the value of the hand if an ace is present.

The next class defined is Chips, which represents the player's chips. It has attributes for the total number of chips the player has and the amount the player has bet. It also has methods for increasing or decreasing the player's total number of chips based on the outcome of a hand.

Finally, the Game class is defined, which represents a game of blackjack. It has attributes for the deck of cards, the visible cards in the game, and the player's chips. It has methods for taking a bet from the player, hitting the player's hand, allowing the player to hit or stand, showing the visible cards in the game, and handling the outcomes of the game.

The code creates an instance of the Deck class, an instance of the Game class, and two instances of the Hand class to represent the player's and dealer's hands. It then deals the initial cards and sets up the visible cards and the player's chips.

The game loop starts with the player being allowed to hit or stand until they either stand or their hand value exceeds 21. If the player's hand value is below 21, the dealer then hits until their hand value is at least 17. Finally, the outcomes of the game are determined and the player's chips are adjusted accordingly.

In [7]:
deck = Deck()
game = Game(deck)

player_hand = Hand()
player_hand.add_card(game.deck.draw_card())
player_hand.add_card(game.deck.draw_card())
game.visibile_cards.extend(player_hand.cards)

dealer_hand = Hand()
dealer_hand.add_card(game.deck.draw_card())
dealer_hand.add_card(game.deck.draw_card())
game.visibile_cards.append(dealer_hand.cards[0])

player_chips = Chips()
game.take_bet(player_chips)

game.show_some(player_hand, dealer_hand)

global playing
playing = True
while playing:
    game.hit_or_stand(player_hand)
    game.show_some(player_hand, dealer_hand)
    if player_hand.value > 21:
        game.player_busts(player_hand, dealer_hand, player_chips)
        break

if player_hand.value <= 21:
    while dealer_hand.value < 17:
        game.hit(dealer_hand)
    game.show_all(player_hand, dealer_hand)
    if dealer_hand.value > 21:
        game.dealer_busts(player_hand, dealer_hand, player_chips)
    elif dealer_hand.value > player_hand.value:
        game.dealer_wins(player_hand, dealer_hand, player_chips)
    elif dealer_hand.value < player_hand.value:
        game.player_wins(player_hand, dealer_hand, player_chips)
    else:
        game.push(player_hand, dealer_hand)

KeyboardInterrupt: ignored

<div class="markdown-google-sans">

## **Probabilities!**
</div>

Blackjack probabilities are an essential aspect of the game, allowing players to calculate their odds of winning based on the cards they are dealt. One of the key factors in blackjack probability is the number of decks being used, as this affects the chances of drawing specific cards. Understanding the probability of getting a blackjack, busting, or hitting a specific hand value is crucial for making informed decisions during gameplay, such as whether to hit, stand, or double down. By taking the time to learn and apply basic blackjack probability strategies, players can increase their chances of winning and make the most of their time at the table.


The total number of combinations for the cards in blackjack are 52*51/2 = 1326. If we would care about the order of the two cards, the number of combinations would be 2652

The probability of getting a blackjack in the beginning if the card game is: (4/52)*(16/51)/2 = 1.20663%

The probability of getting 20 points in the beginning if the card game is: (4/52)*(4/51)/2 + (16/52)*(15/51)/2 = 4.82655%

The probability of getting 19 points in the beginning if the card game is: (4/52)*(4/51)/2 + (16/52)*(4/51)/2 = 1.5083%

The probability of getting 18 points in the beginning if the card game is: (4/52)*(4/51)/2 + (16/52)*(4/51)/2 = 1.5083%

Let's create a function which calculates the probability to get a good hand in the next move

In [ ]:
def calculate_probablity(player_hand, visible_cards, deck):
    values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}
    deck = [card for card in deck.cards if card not in visible_cards]
    probablity17 = 0
    probablity18 = 0
    probablity19 = 0
    probablity20 = 0
    probablity21 = 0
    probablitybust = 0
    for card in deck:
        if player_hand.value + values[card[0]] == 17:
            probablity17 += 1/len(deck)
        elif player_hand.value + values[card[0]] == 19:
            probablity19 += 1/len(deck)
        elif player_hand.value + values[card[0]] == 19:
            probablity19 += 1/len(deck)
        elif player_hand.value + values[card[0]] == 20:
            probablity20 += 1/len(deck)
        elif player_hand.value + values[card[0]] == 21:
            probablity21 += 1/len(deck)
        elif player_hand.value + values[card[0]] > 21:
            probablitybust += 1/len(deck)
    print(f'Probablity of getting 17: {round(probablity17*100, 4)}%')
    print(f'Probablity of getting 18: {round(probablity18*100, 4)}%')
    print(f'Probablity of getting 19: {round(probablity19*100, 4)}%')
    print(f'Probablity of getting 20: {round(probablity20*100, 4)}%')
    print(f'Probablity of getting 21: {round(probablity21*100, 4)}%')
    print(f'Probablity of getting busted: {round(probablitybust*100, 4)}%')

deck = Deck()
calculate_probablity(player_hand, game.visibile_cards, deck)

Probablity of getting 17: 0%
Probablity of getting 18: 0%
Probablity of getting 19: 6.1224%
Probablity of getting 20: 8.1633%
Probablity of getting 21: 8.1633%
Probablity of getting busted: 69.3878%


Let's create a simulation and calculate the likelihood of the dealer busts depending on their first card.

In [81]:
import numpy as np
import pandas as pd

values = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}
array_1 = list(values.keys())
array_2 = [0, 17, 18, 19, 20, 21]

array = np.zeros([len(array_1), len(array_2)], dtype=float)

for i in range(300000):
  deck = Deck()
  deck.shuffle()
  dealer_hand = Hand()
  dealer_hand.add_card(deck.draw_card())
  dealer_hand.add_card(deck.draw_card())
  inital_card_value = dealer_hand.cards[0][0]

  while dealer_hand.value < 17:
      dealer_hand.add_card(deck.draw_card())
      dealer_hand.adjust_for_ace()
  final_value = dealer_hand.value
  if final_value > 21:
    final_value = 0

  y = array_2.index(final_value)
  x = array_1.index(inital_card_value)

  array[x][y] += 1

percentages = array.sum(axis=1)
for i, row in enumerate(np.copy(array)):
  for j, cell in enumerate(row):
    cell = cell/percentages[i] * 100
    array[i][j] = cell

df = pd.DataFrame(array, columns=array_2, index=array_1)
print(df)

              0          17         18         19         20         21
2      34.865515  13.863984  13.111743  13.234201  12.796851  12.127706
3      37.940933  13.096429  12.674525  12.139533  12.113436  12.035144
4      39.832231  13.045358  11.631426  12.254075  11.726553  11.510356
5      42.739310  11.786412  12.463642  11.816801  10.627306  10.566529
6      42.357664  16.834290  10.752363  10.394095  10.104891   9.556697
7      26.068045  36.744186  14.108527   7.661499   8.006029   7.411714
8      24.344163  12.751911  35.875608  13.346942   6.797255   6.884121
9      23.967945  12.436647  10.318389  35.737492  11.604938   5.934590
10     21.631129  11.681939  11.265037  11.577713  32.644287  11.199896
Jack   21.522979  11.489288  11.070318  11.567035  32.718556  11.631824
Queen  21.606053  11.396269  10.992176  11.477947  33.191471  11.336085
King   21.237015  11.400878  11.231364  11.444343  33.137741  11.548659
Ace    15.543884  11.941141  11.962844  12.175536  12.370866  36

From the table and the simulation above we can see that if the dealer has a face up card of an Jack the likelihood of getting bust during the game is 21.5%